### Extracción de capas de interés de SIOSE
#### (Solar, Cultivos y Cultivos arbóreos)

##### Instalar paquetes no nativos

In [ ]:
!pip install geopandas pyproj

In [ ]:
!pip install fiona

##### Importar paquetes

In [12]:
import os
import glob
import geopandas as gpd
from pyproj import CRS
import zipfile
import fiona
import pandas as pd
import pickle
import pandas as pd

##### Listar todos los archivos de SIOSE, formato .gpkg para cada provincia

In [4]:
# Directorio raíz
directorio_raiz = "spatial_data/"

# Lista para almacenar los archivos .gpkg
gpkg_files = []

# Recorrer recursivamente el directorio raíz y sus subdirectorios
for directorio_actual, _, archivos in os.walk(directorio_raiz):
    # Filtrar los archivos .gpkg y agregarlos a la lista
    gpkg_files += [os.path.join(directorio_actual, archivo) for archivo in archivos if archivo.endswith(".gpkg")]

# Imprimir la lista de archivos .gpkg
print(gpkg_files)

['spatial_data/06_BADAJOZ.gpkg', 'spatial_data/13_CREAL.gpkg']


##### Extracción polígonos solares

In [5]:
# Función para extraer la capa USOS y el uso 'Solar'
def extract_USOS(file_path):
    # Listar todas las capas
    layers = fiona.listlayers(file_path)
    # Seleccionar el nombre de capa que termine por '_T_USOS'
    layer_name = [layer for layer in layers if layer.endswith('_T_USOS')][0]
    # Leer la capa
    layer = gpd.read_file(file_path, layer=layer_name)
    # Extraer uso 'Solar'
    layer = layer[layer['ID_USO_MAX'].isin([2442])]
    return layer

In [7]:
# Extraer capas 
solar_polygons = [extract_USOS(file) for file in gpkg_files]

In [8]:
# Proyectar a EPSG:32630
solar_polygons = [capa.to_crs(32630) for capa in solar_polygons]

In [ ]:
# Iterar sobre las capas de la lista para ver cuáles tienen más polígonos
for i, geo_database in enumerate(solar_polygons, start=1):
    num_polygons = len(geo_database)  # Obtener la cantidad de polígonos en la geodatabase actual
    print(f"La geodatabase {i} contiene {num_polygons} polígonos.")

# La que más tiene es Granada, pero nos quedamos con Badajoz que es la segunda que más tiene
# porque los polígonos de Granada creo que de media son mucho más pequeños

La geodatabase 1 contiene 1 polígonos.
La geodatabase 2 contiene 51 polígonos.
La geodatabase 3 contiene 74 polígonos.
La geodatabase 4 contiene 205 polígonos.
La geodatabase 5 contiene 491 polígonos.
La geodatabase 6 contiene 27 polígonos.
La geodatabase 7 contiene 29 polígonos.
La geodatabase 8 contiene 282 polígonos.
La geodatabase 9 contiene 48 polígonos.
La geodatabase 10 contiene 71 polígonos.
La geodatabase 11 contiene 64 polígonos.
La geodatabase 12 contiene 27 polígonos.
La geodatabase 13 contiene 502 polígonos.
La geodatabase 14 contiene 0 polígonos.
La geodatabase 15 contiene 54 polígonos.
La geodatabase 16 contiene 43 polígonos.
La geodatabase 17 contiene 40 polígonos.
La geodatabase 18 contiene 86 polígonos.
La geodatabase 19 contiene 113 polígonos.
La geodatabase 20 contiene 114 polígonos.
La geodatabase 21 contiene 2 polígonos.
La geodatabase 22 contiene 6 polígonos.
La geodatabase 23 contiene 59 polígonos.
La geodatabase 24 contiene 418 polígonos.
La geodatabase 25 cont

In [33]:
# Guardar variable de Python como archivo (pickle package)

# Ruta del archivo
ruta_archivo = "spatial_data/solar_polygons.pkl"

# Guardar la variable en el archivo
with open(ruta_archivo, "wb") as archivo:
    pickle.dump(solar_polygons, archivo)

print("Variable guardada correctamente en", ruta_archivo)

# Así podemos cargar la variable en un nuevo entorno


Variable guardada correctamente en spatial_data/solar_polygons.pkl


In [ ]:
# En caso de necesitar rehacer proceso podemos volver a cargar la variable
# Abre el archivo en modo lectura binaria
with open('spatial_data/solar_polygons.pkl', 'rb') as file:
    solar_polygons = pickle.load(file)

In [14]:
# Seleccionar Badajoz y Ciudad Real y unir
sub_solar = pd.concat([solar_polygons[4], solar_polygons[11]], ignore_index=True)
print(sub_solar)

     PROVINCIA  MUNICIPIO     MUNICIPIO_NOMBRE TIPO  \
0            6        158                ZAFRA    R   
1            6        162           ZARZA (LA)    R   
2            6        162           ZARZA (LA)    R   
3            6        162           ZARZA (LA)    R   
4            6        162           ZARZA (LA)    R   
..         ...        ...                  ...  ...   
897         13         99         ROBLEDO (EL)    R   
898         13         99         ROBLEDO (EL)    R   
899         13         99         ROBLEDO (EL)    R   
900         13        102  LLANOS DEL CAUDILLO    R   
901         13          1             ABENOJAR    R   

                               ID_PARCELA          REFCAT  \
0    8de429ed-cf83-43e7-9795-d38939db65b6  06158A00700126   
1    10203d9f-229a-4438-8b34-db33e3213cd5  06162A01300026   
2    5951c6d3-4944-4989-8de0-f2b717c47560  06162A01300026   
3    734a0c97-ad1b-4b5e-8669-c670d2c80133  06162A01300026   
4    8bfce0e4-c674-43eb-8444-1753d

In [15]:
# Guardar
sub_solar.to_file("spatial_data/final_polys/badajoz_creal_solar.shp")

C:\Users\Laura Mapu\AppData\Local\Temp\ipykernel_16728\3926020004.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  sub_solar.to_file("spatial_data/final_polys/badajoz_creal_solar.shp")


##### Extracción polígonos invernaderos

In [3]:
# Continuamos sólo con Badajoz para facilitar el proceso
capas_badajoz=fiona.listlayers("spatial_data/06_BADAJOZ.gpkg")
capas_badajoz

['SAR_06_T_USOS',
 'SAR_06_T_POLIGONOS',
 'SAR_06_T_COMBINADA',
 'layer_styles',
 'LISTADO_COBERTURAS',
 'LISTADO_ATRIBUTOS',
 'LISTADO_USOS',
 'SAR_06_T_VALORES',
 'SAR_06_TABLA_PLANA']

In [4]:
# La capa que nos interesa es la de POLIGONOS, donde se encuentran los usos agrícolas detallados según SIGPAC
badajoz_POLIGONOS = gpd.read_file("spatial_data/06_BADAJOZ.gpkg", layer=capas_badajoz[1])

In [48]:
# Nuestros polígonos de interés son los que en la columna de SIGPAC tienen la categoría 'IV'
# Extraer polígonos, proyectar y guardar
badajoz_greenhouse = badajoz_POLIGONOS[badajoz_POLIGONOS['USO_SIGPAC'].isin(['IV'])].to_crs(25830)
badajoz_greenhouse.to_file("spatial_data/final_polys/badajoz_greenhouse.shp")

C:\Users\Laura Mapu\AppData\Local\Temp\ipykernel_1932\1271608959.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  badajoz_greenhouse.to_file("spatial_data/final_polys/badajoz_greenhouse.shp")


##### Extracción polígonos cultivos arbóreos

In [49]:
badajoz_POLIGONOS['USO_SIGPAC'].unique()


array(['PS', 'PR', 'ZU', None, 'TA', 'IM', 'FY', 'TH', 'OV', 'ED', 'PA',
       'AG', 'CA', 'FO', 'OF', 'FS', 'VI', 'VF', 'IV', 'FF', 'VO', 'CI',
       'FL', 'FV', 'CF', 'OC', 'ZC'], dtype=object)

In [51]:
badajoz_treecrops = badajoz_POLIGONOS[badajoz_POLIGONOS['USO_SIGPAC'].isin(['CF', 'CI', 'CS', 'CV', 'FF',
                                                                          'FL', 'FS', 'FV', 'FY', 'OC',
                                                                          'OF', 'OV', 'VF', 'VI', 'VO'])].to_crs(25830)
badajoz_treecrops.to_file("spatial_data/final_polys/badajoz_treecrops.shp")

C:\Users\Laura Mapu\AppData\Local\Temp\ipykernel_1932\2533707338.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  badajoz_treecrops.to_file("spatial_data/final_polys/badajoz_treecrops.shp")


##### Extracción polígonos urbanos

In [6]:
# Nuestra capa de interés es COMBINADA
badajoz_COMBINADA = gpd.read_file("spatial_data/06_BADAJOZ.gpkg", layer=capas_badajoz[2])

In [7]:
# Seleccionamos los polígonos cuya 'COBERTURA_DESC' sea 'Edificación'
badajoz_urban = badajoz_COMBINADA[badajoz_COMBINADA['COBERTURA_DESC'].isin(['Edificación'])].to_crs(25830)
badajoz_urban.to_file("spatial_data/final_polys/badajoz_urban.shp")

C:\Users\Laura Mapu\AppData\Local\Temp\ipykernel_3376\1663385929.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  badajoz_urban.to_file("spatial_data/final_polys/badajoz_urban.shp")


##### Extracción todos los demás polígonos

In [ ]:
# esto no hace falta hacerlo porque realmente es asignar la categoría 'otros' a todos los píxeles
# que no correspondan a ninguna de las anteriores así que ya está!!